# 1. Loading libs

In [ ]:
# importing required libraries
import re
import nltk
from nltk.corpus import movie_reviews
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('movie_reviews')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# 2. preprocessing(punctuations, stopwords, lemmatization)

In [ ]:
print(len(movie_reviews.fileids()))
movie_reviews.categories()

2000


['neg', 'pos']

In [ ]:
print(len(movie_reviews.fileids('pos')))
print(len(movie_reviews.fileids('neg')))

1000
1000


In [ ]:
# Document is a list of (words of review, category of review)
document = [(movie_reviews.words(file_id),category) for file_id in movie_reviews.fileids() for category in movie_reviews.categories(file_id)]
document[:10]

[(['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...], 'neg'),
 (['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...], 'neg'),
 (['it', 'is', 'movies', 'like', 'these', 'that', 'make', ...], 'neg'),
 (['"', 'quest', 'for', 'camelot', '"', 'is', 'warner', ...], 'neg'),
 (['synopsis', ':', 'a', 'mentally', 'unstable', 'man', ...], 'neg'),
 (['capsule', ':', 'in', '2176', 'on', 'the', 'planet', ...], 'neg'),
 (['so', 'ask', 'yourself', 'what', '"', '8mm', '"', '(', ...], 'neg'),
 (['that', "'", 's', 'exactly', 'how', 'long', 'the', ...], 'neg'),
 (['call', 'it', 'a', 'road', 'trip', 'for', 'the', ...], 'neg'),
 (['plot', ':', 'a', 'young', 'french', 'boy', 'sees', ...], 'neg')]

In [ ]:
print(document[0][0])
print(document[0][1])

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...]
neg


In [ ]:
len(document)

2000

In [ ]:
from nltk import WordNetLemmatizer
punctuations = string.punctuation
eng_stopwords = stopwords.words('english')

def doLemmatizeWord(text):
    lemmatizer = WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word) for word in text]
    return lemma

document_punc_stopwords_lem = []

for doc in document :
    #print(doc[0])
    words_wo_puncts = [x for x in doc[0] if x not in punctuations]
    #print(len(words_wo_puncts))
    words_wo_puncts_stopwords = [x for x in words_wo_puncts if x not in eng_stopwords]
    #print(words_wo_puncts_stopwords)
    didWordNetLem = doLemmatizeWord(words_wo_puncts_stopwords)
    sentiment = 0
    if (doc[1]=='neg') :
        sentiment = 1
    document_punc_stopwords_lem.append((didWordNetLem, sentiment))


In [ ]:
import pandas as pd

df = pd.DataFrame(document_punc_stopwords_lem, columns =['Review', 'Sentiment'])
df['Final']=df['Review'].apply(lambda x: ''.join(i+' ' for i in x))
df.head()

,Review,Sentiment,Final
0,"[plot, two, teen, couple, go, church, party, d...",1,plot two teen couple go church party drink dri...
1,"[happy, bastard, quick, movie, review, damn, y...",1,happy bastard quick movie review damn y2k bug ...
2,"[movie, like, make, jaded, movie, viewer, than...",1,movie like make jaded movie viewer thankful in...
3,"[quest, camelot, warner, bros, first, feature,...",1,quest camelot warner bros first feature length...
4,"[synopsis, mentally, unstable, man, undergoing...",1,synopsis mentally unstable man undergoing psyc...


# 3. CountVectorizer -> unigrams bigrams

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
'''
 For example an ngram_range of (1, 1) means only unigrams,
 (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams.
 Only applies if analyzer is not callable.
'''

uni_vectorizer = CountVectorizer(ngram_range=(1,1))
cv_df_uni = uni_vectorizer.fit_transform(df['Final'])
print(cv_df_uni.shape)


bi_vectorizer = CountVectorizer(ngram_range=(2,2))
cv_df_bi = bi_vectorizer.fit_transform(df['Final'])
print(cv_df_bi.shape)

(2000, 35099)
(2000, 509148)


# 4. TF-IDF  -> unigrams bigrams

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

uni_tfvectorizer = TfidfVectorizer(ngram_range = (1,1))
tfidf_df_uni = uni_tfvectorizer.fit_transform(df['Final'])
print(tfidf_df_uni.shape)

bi_tfvectorizer = TfidfVectorizer(ngram_range = (2,2))
tfidf_df_bi = bi_tfvectorizer.fit_transform(df['Final'])
print(tfidf_df_bi.shape)


(2000, 35099)
(2000, 509148)


# 5. MultinomialNB


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:

cv_uni_train_X, cv_uni_test_X, cv_uni_train_Y, cv_uni_test_y = train_test_split(cv_df_uni, df['Sentiment'], test_size=0.30, random_state=17)
cv_bi_train_X, cv_bi_test_X, cv_bi_train_Y, cv_bi_test_y = train_test_split(cv_df_bi, df['Sentiment'], test_size=0.30, random_state=17)
tfidf_uni_train_X, tfidf_uni_test_X, tfidf_uni_train_Y, tfidf_uni_test_y = train_test_split(tfidf_df_uni, df['Sentiment'], test_size=0.30, random_state=17)
tfidf_bi_train_X, tfidf_bi_test_X, tfidf_bi_train_Y, tfidf_bi_test_y = train_test_split(tfidf_df_bi, df['Sentiment'], test_size=0.30, random_state=17)


## 5.1 CountVectorizer Unigrams

In [ ]:
clf_uni_cv = MultinomialNB()
clf_uni_cv.fit(cv_uni_train_X, cv_uni_train_Y)
#MultinomialNB()
cv_uni_test_y_pre= clf_uni_cv.predict(cv_uni_test_X)

print('accuracy_score: %.3f' % accuracy_score(cv_uni_test_y, cv_uni_test_y_pre))

accuracy_score: 0.833


## 5.2 CountVectorizer Bigrams

In [ ]:
clf_bi_cv = MultinomialNB()
clf_bi_cv.fit(cv_bi_train_X, cv_bi_train_Y)
#MultinomialNB()
cv_bi_test_y_pre= clf_bi_cv.predict(cv_bi_test_X)

print('accuracy_score: %.3f' % accuracy_score(cv_bi_test_y, cv_bi_test_y_pre))

accuracy_score: 0.747


## 5.3 Tf-Idf Unigrams

In [ ]:
clf_uni_tfidf = MultinomialNB()
clf_uni_tfidf.fit(tfidf_uni_train_X, tfidf_uni_train_Y)
#MultinomialNB()
tfidf_uni_test_y_pre= clf_uni_tfidf.predict(tfidf_uni_test_X)

print('accuracy_score: %.3f' % accuracy_score(tfidf_uni_test_y, tfidf_uni_test_y_pre))

accuracy_score: 0.785


## 5.4 TF-IDF Bigrams

In [ ]:
clf_bi_tfidf = MultinomialNB()
clf_bi_tfidf.fit(tfidf_bi_train_X, tfidf_bi_train_Y)
#MultinomialNB()
tfidf_bi_test_y_pre= clf_bi_tfidf.predict(tfidf_bi_test_X)

print('accuracy_score: %.3f' % accuracy_score(tfidf_bi_test_y, tfidf_bi_test_y_pre))

accuracy_score: 0.752


## 6. Use nltk.collocations functions to calculate the top 10 bigram collocations based on PPMI

In [ ]:
from nltk.collocations import *
import nltk
from nltk.util import ngrams
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

In [ ]:
len(df['Final'].values.tolist())

2000

In [ ]:
'''
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(df['Final'].values.tolist())
finder.nbest(bigram_measures.pmi, 10)
'''

In [ ]:
words_wo_puncts = [x for x in movie_reviews.words() if x not in punctuations]
words_wo_puncts_stopwords = [x for x in words_wo_puncts if x not in eng_stopwords]
didWordNetLem = doLemmatizeWord(words_wo_puncts_stopwords)

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(didWordNetLem)
finder.apply_freq_filter(3)
finder.nbest(bigram_measures.pmi, 10)

[('_october', 'sky_'),
 ('_patlabor', 'movie_'),
 ('alanis', 'morissette'),
 ('arija', 'bareikis'),
 ('bokeem', 'woodbine'),
 ('edie', 'mcclurg'),
 ('gabriella', 'pescucci'),
 ('gedde', 'watanabe'),
 ('heath', 'ledger'),
 ('igor', 'stravinsky')]

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(movie_reviews.words())
finder.apply_freq_filter(3)
finder.nbest(bigram_measures.pmi, 10)

[('_october', 'sky_'),
 ('alanis', 'morissette'),
 ('arija', 'bareikis'),
 ('battleship', 'potemkin'),
 ('bokeem', 'woodbine'),
 ('edie', 'mcclurg'),
 ('gabriella', 'pescucci'),
 ('gedde', 'watanabe'),
 ('heath', 'ledger'),
 ('igor', 'stravinsky')]

## 7. TopicModeling

In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 32.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=3f7895f399171d2e86ea5278ef35f42b62bc2d5fd6fc80cd614db1fd15a12aca
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=55c1e4d1cbaf38f6d4974e008db83f18bb3efee42fb9e515e8b2c0b74421f8ce
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyLDAvis sklearn


In [ ]:
from __future__ import print_function
import pyLDAvis

import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
# Load 20 newsgroups dataset
# First, the 20 newsgroups dataset available in sklearn is loaded. As always, the headers, footers and quotes are removed.

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [ ]:
newsgroups = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))
docs_raw = newsgroups.data
print(len(docs_raw))

11314


In [ ]:
categories = ['sci.med', 'sci.space', 'talk.politics.guns']
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=categories)

In [ ]:
print(newsgroups_train.keys())
print(newsgroups_train['target_names'])

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])
['sci.med', 'sci.space', 'talk.politics.guns']


In [ ]:
from pprint import pprint
pprint(list(newsgroups_train.target_names))
pprint(list(newsgroups_test.target_names))

['sci.med', 'sci.space', 'talk.politics.guns']
['sci.med', 'sci.space', 'talk.politics.guns']


In [ ]:
print(len(newsgroups_train['data']))
print(len(newsgroups_train['target']))

print(len(newsgroups_test['data']))
print(len(newsgroups_test['target']))

1733
1733
1154
1154


Row Data

In [ ]:
docs_raw_train = newsgroups_train['data']
docs_raw_train[:2]

['\n\nThere are, but not any that would help Texans: In many states,\nsuch laws have been found to violate the state constitution. \nBut the federal Second Amendment does not apply directly to the\nstates. It was written to limit the federal government only. \nThe Fourteenth Amendment was written to extend the restrictions\nof the Bill of Rights to the state level. However, the exact\nwording of the Fourteenth Amendment is very vague. The Supreme\nCourt has been dancing around the issue without facing it\ndirectly for over 100 years. In practice, the Bill of Right\n(indirectly applies through the Fourteenth) applies to the\nstate governments only if the Supreme Court has ruled that \nparticular provision. The Court has made no such rulings on\nthe Second Amendment.',
 " A cash award is OK. A time limit would be nice. You can't give away\nmining rights (assuming there's anything to mine) because you don't own\nthem.\n -----------------------------------------------------------------\n .

Lowercase

In [ ]:
raw_data = newsgroups_train['data']
lower_raw_data = []

for row in raw_data:
  lower_raw_data.append(row.lower())

lower_raw_data[:2]

['\n\nthere are, but not any that would help texans: in many states,\nsuch laws have been found to violate the state constitution. \nbut the federal second amendment does not apply directly to the\nstates. it was written to limit the federal government only. \nthe fourteenth amendment was written to extend the restrictions\nof the bill of rights to the state level. however, the exact\nwording of the fourteenth amendment is very vague. the supreme\ncourt has been dancing around the issue without facing it\ndirectly for over 100 years. in practice, the bill of right\n(indirectly applies through the fourteenth) applies to the\nstate governments only if the supreme court has ruled that \nparticular provision. the court has made no such rulings on\nthe second amendment.',
 " a cash award is ok. a time limit would be nice. you can't give away\nmining rights (assuming there's anything to mine) because you don't own\nthem.\n -----------------------------------------------------------------\n .

remove html/uml

In [ ]:
#regex = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")

def findRegex(pattern) :
  regex = re.compile(pattern)
  i = 0
  for row in lower_raw_data:
    result = regex.findall(row)
    if (len(result)!=0) :
      print(i)
      display(result)
      break;
    i = i+1

pattern = r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});"
findRegex(pattern)

2


['']

In [ ]:
lower_raw_data[2]

'\nno one else seems to know, so i\'ll post this.\n\nthis topic came up on sci.physics.fusion shortly after the cold-fusion\nflap started.  as i recall, its been done to some experimental mice.\nthey showed various ill effects and eventually died.  the reason is\nthat deuterium does not have exactly the same reaction rates as\nhydrogen due to its extra mass (which causes lower velocity, boltzman\nconstant, mumble).  this throws various bits of body biochemistry out\nof kilter, and you get sick and die.\n\ni\'ve never heard of anyone being poisened this way, in or out of real\nlife.  the process takes quite a while.  if anyone wants to write this\nbook, i would imagine you would have to:\n\n1: replace a significant fraction of the water in the body with heavy\n   water.\n\n2: wait while normal breakdown and repair processes cause other\n   molecules in the body to be synthesised using the deuterium.\n\nduring this process the victim would gradually deteriorate and\neventually die, but i

In [ ]:
uml_html_lower_raw_data = []

for row in lower_raw_data:
  html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
  html_row = re.sub(html, "", row)
  uml_html_row = re.sub(r"https?://\S+|www\.\S+", "",html_row)
  uml_html_lower_raw_data.append(uml_html_row)

uml_html_lower_raw_data[2]

'\nno one else seems to know, so i\'ll post this.\n\nthis topic came up on sci.physics.fusion shortly after the cold-fusion\nflap started.  as i recall, its been done to some experimental mice.\nthey showed various ill effects and eventually died.  the reason is\nthat deuterium does not have exactly the same reaction rates as\nhydrogen due to its extra mass (which causes lower velocity, boltzman\nconstant, mumble).  this throws various bits of body biochemistry out\nof kilter, and you get sick and die.\n\ni\'ve never heard of anyone being poisened this way, in or out of real\nlife.  the process takes quite a while.  if anyone wants to write this\nbook, i would imagine you would have to:\n\n1: replace a significant fraction of the water in the body with heavy\n   water.\n\n2: wait while normal breakdown and repair processes cause other\n   molecules in the body to be synthesised using the deuterium.\n\nduring this process the victim would gradually deteriorate and\neventually die, but i

remove asciicode

In [ ]:
pattern = r'[^\x00-\x7f]'
findRegex(pattern)

665


['ÿ']

In [ ]:
uml_html_lower_raw_data[665]

'hello,  i am not sure if this is the right conference to ask this\nquestion, however, here i go..  i am a commercial fisherman and i \nfell about 3 weeks ago down into the hold of the boat and broke or\ncracked a rib and wrenched and bruised my back and left arm.\n  my question,  i have been to a doctor and was told that it was \nbest to do nothing and it would heal up with no long term effect, and \nindeed i am about 60 % better, however, the work i do is very \nhard and i am still not able to go back to work.  the thing that worries me\nis the movement or "clunking" i feel and hear back there when i move \ncertain ways...  i heard some one talking about the rib they broke \nyears ago and that it still bothers them.ÿ.  any opinions?\nthanx and cheers'

In [ ]:
ascii_uml_html_lower_raw_data = []

for row in uml_html_lower_raw_data:
  ascii_uml_html_lower_raw_data.append(re.sub(r'[^\x00-\x7f]', "", row))

ascii_uml_html_lower_raw_data[665]

'hello,  i am not sure if this is the right conference to ask this\nquestion, however, here i go..  i am a commercial fisherman and i \nfell about 3 weeks ago down into the hold of the boat and broke or\ncracked a rib and wrenched and bruised my back and left arm.\n  my question,  i have been to a doctor and was told that it was \nbest to do nothing and it would heal up with no long term effect, and \nindeed i am about 60 % better, however, the work i do is very \nhard and i am still not able to go back to work.  the thing that worries me\nis the movement or "clunking" i feel and hear back there when i move \ncertain ways...  i heard some one talking about the rib they broke \nyears ago and that it still bothers them..  any opinions?\nthanx and cheers'

remove punctuations

In [ ]:
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
puncts_ascii_uml_html_lower_raw_data = []
# = [x for x in ascii_uml_html_lower_raw_data if x not in punctuations]

for row in ascii_uml_html_lower_raw_data:
  pattern = '[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'
  puncts_ascii_uml_html_lower_raw_data.append(re.sub(pattern, "",row))

puncts_ascii_uml_html_lower_raw_data[:2]

['\n\nthere are but not any that would help texans in many states\nsuch laws have been found to violate the state constitution \nbut the federal second amendment does not apply directly to the\nstates it was written to limit the federal government only \nthe fourteenth amendment was written to extend the restrictions\nof the bill of rights to the state level however the exact\nwording of the fourteenth amendment is very vague the supreme\ncourt has been dancing around the issue without facing it\ndirectly for over 100 years in practice the bill of right\nindirectly applies through the fourteenth applies to the\nstate governments only if the supreme court has ruled that \nparticular provision the court has made no such rulings on\nthe second amendment',
 ' a cash award is ok a time limit would be nice you cant give away\nmining rights assuming theres anything to mine because you dont own\nthem\n \n sig files are like strings  every yoyos got one']

remove escape \n \t

In [ ]:
line = "own\nthem\n \n sig files \tare \tlike st"
display(re.sub('[\\n\\t]', "",line))


'ownthem  sig files are like st'

In [ ]:
escape_puncts_ascii_uml_html_lower_raw_data = []

for row in puncts_ascii_uml_html_lower_raw_data:
  escape_puncts_ascii_uml_html_lower_raw_data.append(re.sub('[\\n\\t]', "",row))

escape_puncts_ascii_uml_html_lower_raw_data[:2]

['there are but not any that would help texans in many statessuch laws have been found to violate the state constitution but the federal second amendment does not apply directly to thestates it was written to limit the federal government only the fourteenth amendment was written to extend the restrictionsof the bill of rights to the state level however the exactwording of the fourteenth amendment is very vague the supremecourt has been dancing around the issue without facing itdirectly for over 100 years in practice the bill of rightindirectly applies through the fourteenth applies to thestate governments only if the supreme court has ruled that particular provision the court has made no such rulings onthe second amendment',
 ' a cash award is ok a time limit would be nice you cant give awaymining rights assuming theres anything to mine because you dont ownthem  sig files are like strings  every yoyos got one']

remove stopwords, tokenizer

In [ ]:
#stopwords_escape_puncts_ascii_uml_html_lower_raw_data = [x for x in escape_puncts_ascii_uml_html_lower_raw_data if x not in eng_stopwords]

df_newsgroups_train = pd.DataFrame(escape_puncts_ascii_uml_html_lower_raw_data, columns =['CleanText'])
df_newsgroups_train.head(10)

df_newsgroups_train['TokenizedText'] = df_newsgroups_train['CleanText'].apply(nltk.word_tokenize)
df_newsgroups_train.head()

,CleanText,TokenizedText
0,there are but not any that would help texans i...,"[there, are, but, not, any, that, would, help,..."
1,a cash award is ok a time limit would be nice...,"[a, cash, award, is, ok, a, time, limit, would..."
2,no one else seems to know so ill post thisthis...,"[no, one, else, seems, to, know, so, ill, post..."
3,dcx as is today isnt suitable for this however...,"[dcx, as, is, today, isnt, suitable, for, this..."
4,yes i remember that now well in that case the...,"[yes, i, remember, that, now, well, in, that, ..."


In [ ]:
stop = set(stopwords.words('english'))
df_newsgroups_train['RemoveStopWords'] = df_newsgroups_train['TokenizedText'].apply(lambda x: [word for word in x if word not in stop])
df_newsgroups_train.head()


,CleanText,TokenizedText,RemoveStopWords
0,there are but not any that would help texans i...,"[there, are, but, not, any, that, would, help,...","[would, help, texans, many, statessuch, laws, ..."
1,a cash award is ok a time limit would be nice...,"[a, cash, award, is, ok, a, time, limit, would...","[cash, award, ok, time, limit, would, nice, ca..."
2,no one else seems to know so ill post thisthis...,"[no, one, else, seems, to, know, so, ill, post...","[one, else, seems, know, ill, post, thisthis, ..."
3,dcx as is today isnt suitable for this however...,"[dcx, as, is, today, isnt, suitable, for, this...","[dcx, today, isnt, suitable, however, followon..."
4,yes i remember that now well in that case the...,"[yes, i, remember, that, now, well, in, that, ...","[yes, remember, well, case, cones, indeedcolor..."


LemmatizedText

In [ ]:
df_newsgroups_train['LemmatizedText'] = df_newsgroups_train['RemoveStopWords'].apply(lambda x: doLemmatizeWord(x))
df_newsgroups_train.head()


,CleanText,TokenizedText,RemoveStopWords,LemmatizedText
0,there are but not any that would help texans i...,"[there, are, but, not, any, that, would, help,...","[would, help, texans, many, statessuch, laws, ...","[would, help, texan, many, statessuch, law, fo..."
1,a cash award is ok a time limit would be nice...,"[a, cash, award, is, ok, a, time, limit, would...","[cash, award, ok, time, limit, would, nice, ca...","[cash, award, ok, time, limit, would, nice, ca..."
2,no one else seems to know so ill post thisthis...,"[no, one, else, seems, to, know, so, ill, post...","[one, else, seems, know, ill, post, thisthis, ...","[one, else, seems, know, ill, post, thisthis, ..."
3,dcx as is today isnt suitable for this however...,"[dcx, as, is, today, isnt, suitable, for, this...","[dcx, today, isnt, suitable, however, followon...","[dcx, today, isnt, suitable, however, followon..."
4,yes i remember that now well in that case the...,"[yes, i, remember, that, now, well, in, that, ...","[yes, remember, well, case, cones, indeedcolor...","[yes, remember, well, case, cone, indeedcolor,..."


In [ ]:
df_newsgroups_train['FINAL']=df_newsgroups_train['LemmatizedText'].apply(lambda x: ''.join(i+' ' for i in x))
df_newsgroups_train.head()

,CleanText,TokenizedText,RemoveStopWords,LemmatizedText,FINAL
0,there are but not any that would help texans i...,"[there, are, but, not, any, that, would, help,...","[would, help, texans, many, statessuch, laws, ...","[would, help, texan, many, statessuch, law, fo...",would help texan many statessuch law found vio...
1,a cash award is ok a time limit would be nice...,"[a, cash, award, is, ok, a, time, limit, would...","[cash, award, ok, time, limit, would, nice, ca...","[cash, award, ok, time, limit, would, nice, ca...",cash award ok time limit would nice cant give ...
2,no one else seems to know so ill post thisthis...,"[no, one, else, seems, to, know, so, ill, post...","[one, else, seems, know, ill, post, thisthis, ...","[one, else, seems, know, ill, post, thisthis, ...",one else seems know ill post thisthis topic ca...
3,dcx as is today isnt suitable for this however...,"[dcx, as, is, today, isnt, suitable, for, this...","[dcx, today, isnt, suitable, however, followon...","[dcx, today, isnt, suitable, however, followon...",dcx today isnt suitable however followon sdiof...
4,yes i remember that now well in that case the...,"[yes, i, remember, that, now, well, in, that, ...","[yes, remember, well, case, cones, indeedcolor...","[yes, remember, well, case, cone, indeedcolor,...",yes remember well case cone indeedcolor sensit...


CountVectorizer

In [ ]:
# Convert to document-term matrix
# Next, the raw documents are converted into document-term matrix, possibly as raw counts or in TF-IDF form.

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5,
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(df_newsgroups_train['FINAL'])
print(dtm_tf.shape)

(1733, 2156)


TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(df_newsgroups_train['FINAL'])
print(dtm_tfidf.shape)

(1733, 2156)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:2032: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning,


In [ ]:
# Fit Latent Dirichlet Allocation models - this step takes a while!

# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)

# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

'''
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_components=20, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)'''


"\nLatentDirichletAllocation(batch_size=128, doc_topic_prior=None,\n             evaluate_every=-1, learning_decay=0.7,\n             learning_method='online', learning_offset=10.0,\n             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,\n             n_jobs=1, n_components=20, perp_tol=0.1, random_state=0,\n             topic_word_prior=None, total_samples=1000000.0, verbose=0)"

In [ ]:
# Visualizing the models with pyLDAvis, using the term frequency (tf) vectors
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.143244  0.081060       1        1  8.520181
19     0.223659  0.000591       2        1  8.361900
4     -0.150413  0.090788       3        1  8.045126
9     -0.014424 -0.223863       4        1  7.539438
12    -0.075947 -0.072558       5        1  6.571405
10     0.173858  0.088656       6        1  6.061119
18     0.108573 -0.159173       7        1  5.881252
0     -0.073400  0.046063       8        1  5.512485
3      0.155985 -0.104590       9        1  5.222249
11    -0.126905  0.072079      10        1  5.186087
16     0.190424  0.032861      11        1  4.656196
8     -0.112904 -0.151620      12        1  4.603518
14    -0.152443 -0.042654      13        1  4.598166
5      0.249584  0.086612      14        1  3.701547
17    -0.020705  0.126192      15        1  3.521399
7      0.106480  0.094028      16        1  3.394698
15    -0.117759  0.065315      17        1  2.686219
1     -0.052669 -0.130264      18        1  2.638322
2     -0.061418 -0.006091      19        1  1.877136
6     -0.106335  0.106568      20        1  1.421556, topic_info=         Term        Freq       Total Category  logprob  loglift
837       gun  647.000000  647.000000  Default  30.0000  30.0000
1807    space  742.000000  742.000000  Default  29.0000  29.0000
1045   launch  291.000000  291.000000  Default  28.0000  28.0000
751      food  174.000000  174.000000  Default  27.0000  27.0000
731   firearm  264.000000  264.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
2126     work    6.395940  261.302684  Topic20  -5.3748   0.5434
812      good    6.581943  312.550102  Topic20  -5.3461   0.3930
1026     know    6.644815  455.269571  Topic20  -5.3366   0.0264
1115      lot    5.741808  178.044109  Topic20  -5.4827   0.8192
191   believe    5.637122  185.182430  Topic20  -5.5011   0.7614

[1118 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
4         1  0.162422  accept
4         3  0.609083  accept
4         4  0.040606  accept
4         9  0.081211  accept
4        10  0.040606  accept
...     ...       ...     ...
2154     13  0.211422   youre
2154     14  0.025371   youre
2154     15  0.059198   youre
2154     16  0.109940   youre
2154     17  0.101483   youre

[5215 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 20, 5, 10, 13, 11, 19, 1, 4, 12, 17, 9, 15, 6, 18, 8, 16, 2, 3, 7])

In [ ]:
# Visualizing the models with pyLDAvis, using the tf-idf vectors
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19     0.277056  0.070483       1        1  47.031496
12    -0.145336  0.224164       2        1   9.368495
9     -0.137073  0.065214       3        1   5.962015
16    -0.096450 -0.159437       4        1   4.994772
15    -0.054871 -0.056578       5        1   3.613705
2     -0.061716 -0.056589       6        1   3.433299
10    -0.017458 -0.031322       7        1   3.065155
6      0.061608 -0.022139       8        1   2.779454
18    -0.015497 -0.041875       9        1   2.751848
0     -0.002250 -0.000231      10        1   2.345093
4      0.005820  0.001524      11        1   1.666551
13     0.032157  0.003892      12        1   1.661014
11     0.015281 -0.011149      13        1   1.656825
5      0.004985  0.006790      14        1   1.556426
17     0.034803 -0.003934      15        1   1.511700
3      0.027335  0.003222      16        1   1.441028
14     0.004204 -0.007442      17        1   1.380626
7      0.017945  0.009659      18        1   1.301414
1      0.036983  0.000863      19        1   1.246575
8      0.012476  0.004886      20        1   1.232508, topic_info=             Term       Freq      Total Category  logprob  loglift
1026         know  30.000000  30.000000  Default  30.0000  30.0000
1800         soon  12.000000  12.000000  Default  29.0000  29.0000
751          food  13.000000  13.000000  Default  28.0000  28.0000
813        gordon  12.000000  12.000000  Default  27.0000  27.0000
174          bank  12.000000  12.000000  Default  26.0000  26.0000
...           ...        ...        ...      ...      ...      ...
1260         news   0.673163   8.385944  Topic20  -4.9807   1.8738
103   appreciated   0.379729   3.786763  Topic20  -5.5533   2.0963
1029        known   0.449334   6.743208  Topic20  -5.3850   1.6876
184          bear   0.390744   4.564205  Topic20  -5.5247   1.9382
1682         said   0.492510  13.862381  Topic20  -5.2932   1.0587

[919 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
6         1  0.655400   accepted
6        18  0.327700   accepted
13        4  0.729618       acid
13       13  0.243206       acid
14        1  0.650321        act
...     ...       ...        ...
2149      5  0.360991  yesterday
2150      9  0.606134       york
2152      4  0.303658      youll
2152      5  0.303658      youll
2152      8  0.303658      youll

[1211 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 13, 10, 17, 16, 3, 11, 7, 19, 1, 5, 14, 12, 6, 18, 4, 15, 8, 2, 9])

Compared to Componenet5

In [ ]:
lda_tf_c5 = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tf_c5.fit(dtm_tf)

# for TFIDF DTM
lda_tfidf_c5 = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tfidf_c5.fit(dtm_tfidf)

'''
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_components=20, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)'''


"\nLatentDirichletAllocation(batch_size=128, doc_topic_prior=None,\n             evaluate_every=-1, learning_decay=0.7,\n             learning_method='online', learning_offset=10.0,\n             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,\n             n_jobs=1, n_components=20, perp_tol=0.1, random_state=0,\n             topic_word_prior=None, total_samples=1000000.0, verbose=0)"

In [ ]:
# Visualizing the models with pyLDAvis, using the term frequency (tf) vectors
pyLDAvis.sklearn.prepare(lda_tf_c5, dtm_tf, tf_vectorizer)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.149465  0.094913       1        1  27.710175
2      0.185762  0.113150       2        1  19.979360
1     -0.070090 -0.180918       3        1  18.825747
0     -0.109064  0.062065       4        1  16.958590
3      0.142858 -0.089210       5        1  16.526128, topic_info=         Term        Freq       Total Category  logprob  loglift
1807    space  745.000000  745.000000  Default  30.0000  30.0000
837       gun  645.000000  645.000000  Default  29.0000  29.0000
1045   launch  288.000000  288.000000  Default  28.0000  28.0000
731   firearm  262.000000  262.000000  Default  27.0000  27.0000
2093   weapon  265.000000  265.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
1700  science   79.913205  224.199022   Topic5  -5.3027   0.7686
540   disease   75.275676  193.459931   Topic5  -5.3625   0.8563
1515   public   73.076359  179.130210   Topic5  -5.3921   0.9036
1866    study   83.183087  264.608533   Topic5  -5.2626   0.6430
2146     year   95.852018  509.222674   Topic5  -5.1208   0.1301

[349 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
13        2  0.042285      acid
13        4  0.930270      acid
20        1  0.687654  actually
20        2  0.015281  actually
20        3  0.106968  actually
...     ...       ...       ...
2146      2  0.239581      year
2146      3  0.168885      year
2146      4  0.139428      year
2146      5  0.188523      year
2147      4  0.980848     yeast

[730 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 1, 4])

In [ ]:
# Visualizing the models with pyLDAvis, using the tf-idf vectors
pyLDAvis.sklearn.prepare(lda_tfidf_c5, dtm_tfidf, tfidf_vectorizer)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.050308 -0.011694       1        1  44.463296
3     -0.021286 -0.085856       2        1  30.988081
1     -0.112537  0.054908       3        1  11.887281
4      0.010606 -0.029388       4        1   6.348851
2      0.072909  0.072030       5        1   6.312490, topic_info=           Term       Freq      Total Category  logprob  loglift
837         gun  25.000000  25.000000  Default  30.0000  30.0000
1800       soon  11.000000  11.000000  Default  29.0000  29.0000
813      gordon  11.000000  11.000000  Default  28.0000  28.0000
174        bank  11.000000  11.000000  Default  27.0000  27.0000
1899  surrender  10.000000  10.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
1475   probably   1.245363  13.251148   Topic5  -5.9990   0.3980
1115        lot   1.201699  13.606756   Topic5  -6.0347   0.3358
555        dont   1.316792  31.463295   Topic5  -5.9432  -0.4110
879        high   1.143142  10.743797   Topic5  -6.0847   0.5221
1368    patient   1.084357  11.527123   Topic5  -6.1375   0.3989

[286 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
13        1  0.899205    acid
19        2  0.326289  actual
19        4  0.652578  actual
43        1  0.363398  afraid
43        5  0.363398  afraid
...     ...       ...     ...
2146      3  0.046216    year
2147      1  0.923932   yeast
2152      1  0.302166   youll
2152      2  0.302166   youll
2152      5  0.302166   youll

[459 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 5, 3])

Compared 'learning_decay'

In [ ]:
lda_tf_c5_ld01 = LatentDirichletAllocation(n_components=5, random_state=0, learning_decay=0.01)
lda_tf_c5_ld01.fit(dtm_tf)

lda_tf_c5_ld1 = LatentDirichletAllocation(n_components=5, random_state=0, learning_decay=1)
lda_tf_c5_ld1.fit(dtm_tf)

LatentDirichletAllocation(learning_decay=1, n_components=5, random_state=0)

In [ ]:
pyLDAvis.sklearn.prepare(lda_tf_c5_ld01, dtm_tf, tf_vectorizer)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.149465  0.094913       1        1  27.710175
2      0.185762  0.113150       2        1  19.979360
1     -0.070090 -0.180918       3        1  18.825747
0     -0.109064  0.062065       4        1  16.958590
3      0.142858 -0.089210       5        1  16.526128, topic_info=         Term        Freq       Total Category  logprob  loglift
1807    space  745.000000  745.000000  Default  30.0000  30.0000
837       gun  645.000000  645.000000  Default  29.0000  29.0000
1045   launch  288.000000  288.000000  Default  28.0000  28.0000
731   firearm  262.000000  262.000000  Default  27.0000  27.0000
2093   weapon  265.000000  265.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
1700  science   79.913205  224.199022   Topic5  -5.3027   0.7686
540   disease   75.275676  193.459931   Topic5  -5.3625   0.8563
1515   public   73.076359  179.130210   Topic5  -5.3921   0.9036
1866    study   83.183087  264.608533   Topic5  -5.2626   0.6430
2146     year   95.852018  509.222674   Topic5  -5.1208   0.1301

[349 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
13        2  0.042285      acid
13        4  0.930270      acid
20        1  0.687654  actually
20        2  0.015281  actually
20        3  0.106968  actually
...     ...       ...       ...
2146      2  0.239581      year
2146      3  0.168885      year
2146      4  0.139428      year
2146      5  0.188523      year
2147      4  0.980848     yeast

[730 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 1, 4])

In [ ]:
pyLDAvis.sklearn.prepare(lda_tf_c5_ld1, dtm_tf, tf_vectorizer)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.149465  0.094913       1        1  27.710175
2      0.185762  0.113150       2        1  19.979360
1     -0.070090 -0.180918       3        1  18.825747
0     -0.109064  0.062065       4        1  16.958590
3      0.142858 -0.089210       5        1  16.526128, topic_info=         Term        Freq       Total Category  logprob  loglift
1807    space  745.000000  745.000000  Default  30.0000  30.0000
837       gun  645.000000  645.000000  Default  29.0000  29.0000
1045   launch  288.000000  288.000000  Default  28.0000  28.0000
731   firearm  262.000000  262.000000  Default  27.0000  27.0000
2093   weapon  265.000000  265.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
1700  science   79.913205  224.199022   Topic5  -5.3027   0.7686
540   disease   75.275676  193.459931   Topic5  -5.3625   0.8563
1515   public   73.076359  179.130210   Topic5  -5.3921   0.9036
1866    study   83.183087  264.608533   Topic5  -5.2626   0.6430
2146     year   95.852018  509.222674   Topic5  -5.1208   0.1301

[349 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
13        2  0.042285      acid
13        4  0.930270      acid
20        1  0.687654  actually
20        2  0.015281  actually
20        3  0.106968  actually
...     ...       ...       ...
2146      2  0.239581      year
2146      3  0.168885      year
2146      4  0.139428      year
2146      5  0.188523      year
2147      4  0.980848     yeast

[730 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 1, 4])

In [ ]:
lda_tf_example = LatentDirichletAllocation(n_components=5, random_state=0, learning_decay=0.1, max_doc_update_iter=1000)
lda_tf_example.fit(dtm_tf)

'''
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_components=20, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)'''

pyLDAvis.sklearn.prepare(lda_tf_example, dtm_tf, tf_vectorizer)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.147930  0.095486       1        1  27.150786
2      0.183852  0.111045       2        1  20.136437
1     -0.068020 -0.182191       3        1  18.990337
0     -0.111163  0.059729       4        1  16.932512
3      0.143260 -0.084068       5        1  16.789929, topic_info=         Term        Freq       Total Category  logprob  loglift
1807    space  743.000000  743.000000  Default  30.0000  30.0000
837       gun  645.000000  645.000000  Default  29.0000  29.0000
1045   launch  287.000000  287.000000  Default  28.0000  28.0000
731   firearm  263.000000  263.000000  Default  27.0000  27.0000
2093   weapon  265.000000  265.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
540   disease   80.017050  193.710747   Topic5  -5.3172   0.9003
1866    study   86.275960  264.700335   Topic5  -5.2419   0.6633
1515   public   73.608421  179.158357   Topic5  -5.4007   0.8949
1700  science   78.195684  224.131223   Topic5  -5.3403   0.7314
2146     year   96.325488  509.139569   Topic5  -5.1318   0.1194

[344 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
13        2  0.042245       acid
13        4  0.950523       acid
20        1  0.694906   actually
20        2  0.015273   actually
20        3  0.106909   actually
...     ...       ...        ...
2149      2  0.046370  yesterday
2149      3  0.046370  yesterday
2150      2  0.046252       york
2150      3  0.215845       york
2150      5  0.724622       york

[713 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 1, 4])